In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import joblib
import json
import os

class FraudAutoencoderKeras:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = None
        self.threshold = 0.0
        self.feature_columns = []
        
    def prepare_features(self, df):
        """Create better features for fraud detection"""
        features = df.copy()
        
        
        features['amount_to_limit'] = features['amount'] / features['credit_limit']
        features['amount_log'] = np.log1p(features['amount'])
        features['is_large_amount'] = (features['amount'] > features['credit_limit'] * 0.3).astype(float)
        
       
        def get_hour_bin(hour):
            if 6 <= hour < 12: return 0  # Morning
            elif 12 <= hour < 18: return 1  # Afternoon
            elif 18 <= hour < 22: return 2  # Evening
            else: return 3  # Night
        
        features['hour_bin'] = features['hour_of_day'].apply(get_hour_bin)
        features['hour_sin'] = np.sin(2 * np.pi * features['hour_of_day'] / 24)
        features['hour_cos'] = np.cos(2 * np.pi * features['hour_of_day'] / 24)
        features['day_sin'] = np.sin(2 * np.pi * features['day_of_week'] / 7)
        features['day_cos'] = np.cos(2 * np.pi * features['day_of_week'] / 7)
        
        # improve the location and use the harversine 
        def haversine_distance(lat1, lon1, lat2, lon2):
            """ Haversine formula"""
            R = 6371  # Earth radius in kilometers
            
            # Convert to radians
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            
            # Haversine formula
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            c = 2 * np.arcsin(np.sqrt(a))
            
            return R * c
        
        features['merchant_distance_km'] = haversine_distance(
            features['card_lat'].values,
            features['card_lon'].values,
            features['merchant_lat'].values,
            features['merchant_lon'].values
        )
        features['distance_log'] = np.log1p(features['merchant_distance_km'])
        
        # 4. TRANSACTION TYPE FEATURES
        # One-hot encode transaction types (ATM more risky)
        transaction_dummies = pd.get_dummies(features['transaction_type'], prefix='txn')
        features = pd.concat([features, transaction_dummies], axis=1)
        
        # 5. RISK TIME FEATURES
        features['is_high_risk_time'] = ((features['is_night'] == 1) | 
                                        (features['hour_of_day'] < 6) |
                                        (features['hour_of_day'] > 22)).astype(float)
        
        # Select final features
        feature_cols = [
            'amount_to_limit', 'amount_log', 'is_large_amount',
            'hour_bin', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
            'merchant_distance_km', 'distance_log',
            'is_night', 'is_high_risk_time'
        ]
        
        # Add transaction type columns
        txn_cols = [col for col in features.columns if col.startswith('txn_')]
        feature_cols.extend(txn_cols)
        
        # Keep only columns that exist
        available_cols = [col for col in feature_cols if col in features.columns]
        self.feature_columns = available_cols
        
        X = features[available_cols].fillna(0)
        y = features['is_fraud']
        
        print(f"Using {len(available_cols)} features: {available_cols}")
        return X, y
    
    def build_improved_model(self, input_dim):
        """Build better autoencoder architecture"""
        input_layer = layers.Input(shape=(input_dim,))
        
        # Encoder - deeper architecture
        x = layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(input_layer)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        # Bottleneck
        encoded = layers.Dense(16, activation='relu')(x)
        
        # Decoder
        x = layers.Dense(32, activation='relu')(encoded)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)
        
        decoded = layers.Dense(input_dim, activation='linear')(x)
        
        self.model = Model(input_layer, decoded)
        
        # Compile with better optimizer
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.0005),
            loss='mse',
            metrics=['mae']
        )
        
        return self.model
    
    def train_with_validation(self, X_train, y_train, epochs=50, batch_size=128):
        """Train with early stopping and validation - NO ModelCheckpoint"""
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        # Use only normal transactions
        X_train_normal = X_train_scaled[y_train == 0]
        
        print(f"Training on {len(X_train_normal)} normal transactions")
        
        # Train/validation split
        X_train_split, X_val_split = train_test_split(
            X_train_normal, test_size=0.15, random_state=42
        )
        
        
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,  
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=0.00001,
                verbose=1
            )
        ]
        
        # Train
        print("\nStarting training...")
        history = self.model.fit(
            X_train_split, X_train_split,
            validation_data=(X_val_split, X_val_split),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )
        val_reconstructions = self.model.predict(X_val_split, verbose=0)
        val_mse = np.mean(np.power(X_val_split - val_reconstructions, 2), axis=1)
        for percentile in [90, 92, 94, 95, 96, 97, 98]:
            threshold = np.percentile(val_mse, percentile)
            preds = (val_mse > threshold).astype(int)
            fp_rate = preds.mean()  
            if fp_rate < 0.02:  
                self.threshold = threshold
                print(f"Selected threshold at {percentile}th percentile: {threshold:.4f}")
                print(f"False positive rate: {fp_rate:.4f}")
                break
        
        if self.threshold == 0:
            self.threshold = np.percentile(val_mse, 97)  # Default
            print(f"Using default threshold (97th percentile): {self.threshold:.4f}")
        
        return history
    
    def evaluate_model(self, X_test, y_test):
        X_test_scaled = self.scaler.transform(X_test)
        reconstructions = self.model.predict(X_test_scaled, verbose=0)
        mse_scores = np.mean(np.power(X_test_scaled - reconstructions, 2), axis=1)
        results = []
        for percentile in [90, 92, 94, 95, 96, 97, 98]:
            threshold = np.percentile(mse_scores, percentile)
            y_pred = (mse_scores > threshold).astype(int)
            cm = confusion_matrix(y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()
            accuracy = (tp + tn) / (tp + tn + fp + fn)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            results.append({
                'percentile': percentile,
                'threshold': threshold,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'detected': y_pred.sum()
            })
        results_df = pd.DataFrame(results)
        best_idx = results_df['f1'].idxmax()
        best_result = results_df.loc[best_idx]
        
        print(f"\n{'='*60}")
        print(f"BEST THRESHOLD: {best_result['percentile']}th percentile")
        print(f"Threshold value: {best_result['threshold']:.4f}")
        print(f"{'='*60}")
        
        print(f"\nAccuracy:  {best_result['accuracy']:.4f}")
        print(f"Precision: {best_result['precision']:.4f}")
        print(f"Recall:    {best_result['recall']:.4f}")
        print(f"F1-Score:  {best_result['f1']:.4f}")
        print(f"Fraud Detected: {best_result['detected']} / {len(y_test)}")
        self.threshold = best_result['threshold']
        
        return best_result
    
    def save_model(self, base_name='fraud_autoencoder'):
        base_name_with_suffix = f"{base_name}_002"
        keras_path = f'{base_name_with_suffix}.keras'
        self.model.save(keras_path)
        print(f" Saved Keras format: {keras_path}")
        h5_path = f'{base_name_with_suffix}.h5'
        self.model.save(h5_path)
        print(f"Saved H5 format: {h5_path}")
        scaler_path = f'{base_name_with_suffix}_scaler.pkl'
        joblib.dump(self.scaler, scaler_path)
        print(f"Saved scaler: {scaler_path}")
        threshold_path = f'{base_name_with_suffix}_threshold.pkl'
        joblib.dump(self.threshold, threshold_path)
        print(f"Saved threshold: {threshold_path} (value: {self.threshold:.4f})")
        config = {
            'feature_columns': self.feature_columns,
            'input_dim': len(self.feature_columns),
            'threshold': float(self.threshold),
            'model_summary': str(self.model.summary()),
            'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'version': '002'
        }
        
        config_path = f'{base_name_with_suffix}_config.json'
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
        print(f"Saved config: {config_path}")
        model_json = self.model.to_json()
        with open(f'{base_name_with_suffix}_architecture.json', 'w') as f:
            f.write(model_json)
        print(f"Saved architecture: {base_name_with_suffix}_architecture.json")
        weights_path = f'{base_name_with_suffix}.weights.h5'
        self.model.save_weights(weights_path)
        print(f"✅ Saved weights: {weights_path}")

if __name__ == "__main__":
    print("="*70)
    print("Traning)")
    print("="*70)
    
    
    df = pd.read_csv('current.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    print(f"Dataset: {df.shape}")
    print(f"Fraud rate: {df['is_fraud'].mean()*100:.2f}%")
    print(f"Columns: {list(df.columns)}")
    
    # Initialize autoencoder
    autoencoder = FraudAutoencoderKeras()
    
    
    print("\n🔧 Preparing features...")
    X, y = autoencoder.prepare_features(df)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    
    print(f"\n📈 Data split:")
    print(f"  Training: {X_train.shape} (Fraud: {y_train.sum()})")
    print(f"  Testing:  {X_test.shape} (Fraud: {y_test.sum()})")
    
    autoencoder.build_improved_model(input_dim=X_train.shape[1])
    autoencoder.model.summary()
    
   
    history = autoencoder.train_with_validation(X_train, y_train, epochs=50)
    
    
    print("\n📊 Evaluating model performance...")
    best_result = autoencoder.evaluate_model(X_test, y_test)
    
    print("\n💾 Saving trained model (with 002 suffix)...")
    autoencoder.save_model('fraud_detection_model')
    
  
    sample_idx = np.random.randint(0, len(X_test))
    sample = X_test.iloc[[sample_idx]]
    sample_scaled = autoencoder.scaler.transform(sample)
    reconstruction = autoencoder.model.predict(sample_scaled, verbose=0)
    mse = np.mean(np.power(sample_scaled - reconstruction, 2))
    
    print(f"Sample MSE: {mse:.4f}")
    print(f"Threshold: {autoencoder.threshold:.4f}")
    print(f"Is Fraud? {'YES 🚨' if mse > autoencoder.threshold else 'NO ✅'}")
    
    print("\n" + "="*70)
    print("="*70)
   
    print("  1. fraud_detection_model_002.keras  (Recommended for FastAPI)")
    print("  2. fraud_detection_model_002.h5     (Legacy format)")
    print("  3. fraud_detection_model_002_scaler.pkl")
    print("  4. fraud_detection_model_002_threshold.pkl")
    print("  5. fraud_detection_model_002_config.json")
    print("  6. fraud_detection_model_002_architecture.json")
    print("  7. fraud_detection_model_002.weights.h5")
    print("\n📊 Model Performance (002):")
    print(f"  F1-Score: {best_result['f1']:.4f}")
    print(f"  Accuracy: {best_result['accuracy']:.4f}")
    print(f"  Precision: {best_result['precision']:.4f}")
    print(f"  Recall: {best_result['recall']:.4f}")
    
   

🤖 CREDIT CARD FRAUD AUTOENCODER TRAINING (Version 002)

📊 Loading dataset...
Dataset: (100000, 17)
Fraud rate: 13.46%
Columns: ['transaction_id', 'cc_number', 'user_id', 'credit_limit', 'amount', 'transaction_type', 'city', 'card_lat', 'card_lon', 'merchant_lat', 'merchant_lon', 'merchant_name', 'timestamp', 'hour_of_day', 'day_of_week', 'is_night', 'is_fraud']

🔧 Preparing features...
Using 15 features: ['amount_to_limit', 'amount_log', 'is_large_amount', 'hour_bin', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'merchant_distance_km', 'distance_log', 'is_night', 'is_high_risk_time', 'txn_ATM', 'txn_ONLINE', 'txn_POS']

📈 Data split:
  Training: (80000, 15) (Fraud: 10765)
  Testing:  (20000, 15) (Fraud: 2691)

🏗️  Building autoencoder architecture...
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 15)]              0         
                       

C:\Users\rishi\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 dense_15 (Dense)            (None, 32)                544       
                                                                 
 batch_normalization_10 (Ba  (None, 32)                128       
 tchNormalization)                                               
                                                                 
 dropout_10 (Dropout)        (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 64)                2112      
                                                                 
 batch_normalization_11 (Ba  (None, 64)                256       
 tchNormalization)                                               
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 15)                975       
          